# frequency

> Functionality for frequency analysis.

In [ ]:
#| default_exp frequency

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import time
import polars as pl
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| hide
path_to_toy_corpus = '../test-corpora/saved/toy.corpus'
path_to_brown_corpus = '../test-corpora/saved/brown.corpus'

In [ ]:
#| export
class Frequency:
	""" Class for frequency analysis reporting """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| hide
# now working from new vocab data structure
brown = Corpus().load(path_to_brown_corpus)
display(brown.vocab.head(5).collect(engine='streaming'))

rank,token_id,token,frequency_lower,frequency_orth,is_punct,is_space
1,22848,"""the""",63516,62473,false,false
2,8128,""",""",58331,58331,true,false
3,38309,""".""",49907,49907,true,false
4,2739,"""of""",36321,36122,false,false
5,7126,"""and""",27787,27633,false,false


In [ ]:
#| export
@patch
def frequencies(self: Frequency,
				case_insensitive:bool=True, # frequencies for tokens lowercased or with case preserved
				normalize_by:int=1000000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return
				page_current:int=1, # current page
				show_token_id:bool=False, # show token_id in output
				exclude_tokens:list[str]=[], # exclude specific tokens from frequency report, can be used to remove stopwords
				restrict_tokens:list[str]=[], # restrict frequency report to return frequencies for a list of specific tokens
				exclude_punctuation:bool=True, # exclude punctuation tokens
				exclude_spaces:bool=True # exclude space tokens
				) -> Result: # return a Result object with the frequency table
	""" Report frequent tokens. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if case_insensitive:
		frequency_column = 'frequency_lower'
	else:
		frequency_column = 'frequency_orth'

	columns = ['rank', 'token', 'frequency']
	if show_token_id == True:
		columns = ['rank', 'token_id', 'token', 'frequency']

	count_tokens = self.corpus.token_count
	tokens_descriptor = 'all tokens'
	total_descriptor = 'Total tokens'
	if exclude_punctuation and exclude_spaces:
		count_tokens = self.corpus.word_token_count
		tokens_descriptor = 'word tokens'
		total_descriptor = 'Total word tokens'
	elif exclude_punctuation:
		space_tokens_count = self.corpus.spaces.select(pl.len()).collect(engine='streaming').item()
		count_tokens = self.corpus.word_token_count + space_tokens_count
		tokens_descriptor = 'word and space tokens'
		total_descriptor = 'Total word and space tokens'
	elif exclude_spaces:
		punct_tokens_count = self.corpus.puncts.select(pl.len()).collect(engine='streaming').item()
		count_tokens = self.corpus.word_token_count + punct_tokens_count
		tokens_descriptor = 'word and punctuation tokens'
		total_descriptor = 'Total word and punctuation tokens'

	formatted_data = []
	formatted_data.append(f'Report based on {tokens_descriptor}')

	logger.info(f'Frequencies report time: {(time.time() - start_time):.5f} seconds')

	df = self.corpus.vocab.filter(pl.col(frequency_column).is_not_null())
	if exclude_tokens:
		df = df.filter(~pl.col('token').is_in(exclude_tokens))
	if restrict_tokens:
		df = df.filter(pl.col('token').is_in(restrict_tokens))
	if exclude_punctuation:
		df = df.filter(pl.col('is_punct') == False)
	if exclude_spaces:
		df = df.filter(pl.col('is_space') == False)

	df = df.sort(by = frequency_column, descending=True)

	unique_tokens = df.select(pl.len()).collect(engine='streaming').item()

	df = df.slice((page_current-1)*page_size, page_current*page_size).rename({frequency_column: "frequency"}).select(*columns)

	# if a number is passed then normalize by that number
	df = df.with_columns(((pl.col("frequency") / count_tokens) * normalize_by).alias('normalized_frequency'))
	columns.append('normalized_frequency')

	df = df.drop('rank').with_row_index(name='rank', offset=(page_current-1)*page_size+1)

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.append(f'{total_descriptor}: {count_tokens:,.0f}')

	formatted_data.append(f'Unique tokens: {unique_tokens:,.0f}')
	if unique_tokens > page_size:
		formatted_data.append(f'Showing {page_size} rows')
		formatted_data.append(f'Page {page_current} of {unique_tokens // page_size + 1}')

	return Result(type = 'frequencies', df=df.collect(engine='streaming'), title='Frequencies', description='Frequencies of tokens in the corpus', summary_data={}, formatted_data=formatted_data)


In [ ]:
# load the corpus
brown = Corpus().load(path_to_brown_corpus)

In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)

In [ ]:
#| hide
print(toy)
print(brown)

┌────────────────────┬──────────────────────────────────┐
│ Attribute          ┆ Value                            │
╞════════════════════╪══════════════════════════════════╡
│ Name               ┆ Toy Corpus                       │
│ Description        ┆ Toy corpus for testing           │
│ Date Created       ┆ 2025-05-28 14:49:02              │
│ Conc Version       ┆ 0.0.1                            │
│ Corpus Path        ┆ ../test-corpora/saved/toy.corpus │
│ Document Count     ┆ 6                                │
│ Token Count        ┆ 38                               │
│ Word Token Count   ┆ 32                               │
│ Unique Tokens      ┆ 15                               │
│ Unique Word Tokens ┆ 14                               │
└────────────────────┴──────────────────────────────────┘
┌────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# instantiate the Frequency class
report_brown = Frequency(brown)

In [ ]:
#| hide
report_toy = Frequency(toy)
report_toy.frequencies(normalize_by=10000, page_size=20).display()

Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 8 
 2,500.00 
 
 
 2 
 is 
 4 
 1,250.00 
 
 
 3 
 cat 
 3 
 937.50 
 
 
 4 
 dog 
 3 
 937.50 
 
 
 5 
 on 
 2 
 625.00 
 
 
 6 
 sat 
 2 
 625.00 
 
 
 7 
 mat 
 2 
 625.00 
 
 
 8 
 a 
 2 
 625.00 
 
 
 9 
 climbing 
 1 
 312.50 
 
 
 10 
 barking 
 1 
 312.50 
 
 
 11 
 tree 
 1 
 312.50 
 
 
 12 
 meowing 
 1 
 312.50 
 
 
 13 
 digging 
 1 
 312.50 
 
 
 14 
 hole 
 1 
 312.50 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 32 
 


 
 Unique tokens: 14

In [ ]:
# run the frequencies method and display the results
report_brown.frequencies(normalize_by=10000, page_size=20).display()

Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 63,516 
 648.03 
 
 
 2 
 of 
 36,321 
 370.57 
 
 
 3 
 and 
 27,787 
 283.50 
 
 
 4 
 to 
 25,868 
 263.92 
 
 
 5 
 a 
 22,190 
 226.40 
 
 
 6 
 in 
 19,751 
 201.51 
 
 
 7 
 that 
 10,409 
 106.20 
 
 
 8 
 is 
 10,138 
 103.43 
 
 
 9 
 was 
 9,931 
 101.32 
 
 
 10 
 for 
 8,905 
 90.85 
 
 
 11 
 with 
 7,043 
 71.86 
 
 
 12 
 it 
 6,991 
 71.33 
 
 
 13 
 he 
 6,772 
 69.09 
 
 
 14 
 as 
 6,738 
 68.75 
 
 
 15 
 his 
 6,523 
 66.55 
 
 
 16 
 on 
 6,459 
 65.90 
 
 
 17 
 be 
 6,365 
 64.94 
 
 
 18 
 's 
 5,285 
 53.92 
 
 
 19 
 had 
 5,200 
 53.05 
 
 
 20 
 by 
 5,156 
 52.60 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 980,144 
 


 
 Unique tokens: 42,907 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2146

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()